# Big G Express - Data Exploration

## Team: Elden Ring

<img src="https://eldenring.wiki.fextralife.com/file/Elden-Ring/mirel_pastor_of_vow.jpg" alt="PRAISE DOG" style="width:806px;height:600px;"/>

#### PRAISE THE DOG!

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.feature_selection import chi2
from sklearn.impute import SimpleImputer

In [2]:
faults = pd.read_csv('../data/J1939Faults.csv', low_memory=False, parse_dates=['EventTimeStamp', 'LocationTimeStamp']) #index_col='EventTimeStamp'
service_fault = pd.read_excel('../data/Service Fault Codes_1_0_0_167.xlsx')
vehicle_diagnostic = pd.read_csv('../data/VehicleDiagnosticOnboardData.csv')


/Users/tomoumer/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


Few keyponts from questions to Josh Treet: 
- throw 2011 dates and older out, mistake with an integer overflow that took a few days to correct
- any time being able to predict a derate is great (even just a few hours)
- derates are going to be related to emissions conditions
- coolant level codes (and some others) can often flip between on and off
- derate + light continuing to be on, it's the same event (a pulse of it)
- spn + fmi together determine the fault code
- most trucks fairly similar/same (within like 4 years)
- maybe costs about $500 if misspredicted potential derate 

## Exploratory Data Analysis

In [3]:
print(faults.shape)
print(service_fault.shape)
print(vehicle_diagnostic.shape)

(1187335, 20)
(7124, 14)
(12821626, 4)


Faults joins to vehicle_diagnostic with RecordID = FaultID

Also, columns actionDescription and faultValue in the faults are unused.

`faults['actionDescription'].isna().sum()`

We also remove 2169 EquipmentID that have more than 5 characters

In [4]:
faults = (
    faults.drop(['actionDescription', 'faultValue'], axis=1)
    [faults['EquipmentID'].str.len() <= 5]
)

There are three service locations that appear in the dataset. The fault signals might be going on and off there. In order to eliminate those counts, we check if the Latitutde and Longitude coordinates of the truck are within 0.01 units (in both Lat and Long directions) next to a service location. The 0.01 represent, roughly, the distance of a mile.

Doing so, we eliminate 131778 events.

In [5]:
for lat, lon in [(36.0666667, -86.4347222), (35.5883333, -86.4438888), (36.1950, -83.174722)]:
    
    faults = faults.loc[~((abs(lat - faults['Latitude']) <= 0.01) &
                          (abs(lon - faults['Longitude']) <= 0.01))]

Also filter out all erroneous years, 2011 or earlier, (394 lines), caused by an integer error

In [6]:
faults = faults.loc[faults['EventTimeStamp'].dt.year > 2011]

Finally, remove the rows where 'active' column is False -> those represent where an indicator was turned off (506690) rows.

So we end up with 546674 rows in faults.

In [7]:
faults = faults.loc[faults['active'] == True]

## Modifying faults into rolling window

Next, combine the spn and fmi columns together in order to get them ready to one hot encode and use in the rolling window.

> note: need to order by event time stamp in order to use the rolling window later

In [8]:
faults['spn_fmi'] = ['_'.join(i) for i in zip(faults['spn'].astype(str), faults['fmi'].astype(str))]

faults_encoded = pd.get_dummies(faults, columns=['spn_fmi'], prefix='spn_fmi')

faults_encoded = faults_encoded.sort_values(by='EventTimeStamp')

In [9]:
# to obtain the one hot encoded columns since there are so many
spnfmi_cols = [col for col in faults_encoded.columns if 'spn_fmi' in col]
fixed_cols = ['RecordID', 'spn', 'fmi']

In [10]:
# for some reason, the agg function with sum works without grouping by;
# but when added the groupby, it just keeps running without being able to complete

# d1 = dict.fromkeys(fixed_cols, lambda x: x[-1]) #this function gets the last value in group!
# d2 = dict.fromkeys(spnfmi_cols, 'sum')

# d = {**d1, **d2}

# faults_encoded.groupby('EquipmentID')[['EventTimeStamp'] + fixed_cols + spnfmi_cols].rolling(window = '1d', on = "EventTimeStamp").agg(d)

Using the groupby (for each truck) and rolling window on top of that:

In [11]:
faults_rolling = (
    faults_encoded
    .groupby('EquipmentID')[['EventTimeStamp'] + spnfmi_cols]
    .rolling(window = '1d', on = "EventTimeStamp")
    .sum()
)

faults_rolling = faults_rolling.reset_index()

In [12]:
# to bring in back the spn and fmi information - this was an alternative since the arg made the kernel crash!
faults_rolling = pd.merge(faults_encoded[fixed_cols],
                          faults_rolling,
                          left_index= True,
                          right_on = 'level_1').drop(columns='level_1')

randomly sampling 1000 rows of past 24 hrs. we see that the most common fault are:
- 111-17, coolant level below normal, low severity
- 929-9, abnormal update? rate tire location
- 96-3, high voltage in fuel level
- 829-3, high voltage left fuel level
- 596-31, Condition Exists Cruise Control Enable Switch
- 111-18, Low engine coolant level detected, med severity
- 51923-0, ???????????
- 4096-0, High (Severity High) NOx limits exceeded due t....
- 97-15, High (Severity Low) Water In Fuel Indicator; Water has been detected in the fuel filter.
- 639-2, Incorrect Data J1939 Network #1; The ECM has a communication error.
- 629-12, ECM power supply errors / ECM error / ECM data lost
- 2863-7, Not Reporting Data Front Operator Wiper Switch, 
- 1068-2, Incorrect Data Brake Signal Sensor 2, 
- 50353_0, ??????
- 1807_2, Incorrect Data Steering Wheel Angle
- 807_5, Low Current Dif 2 - ASR Valve
- 611_14, Special Instructions System Diagnostic Code #1
- 0_0, ???
- 4276_0, ???
- 412_0, High (Severity High) Engine Exhaust Gas Recirc...; The EGR temperature sensor indicates that the



Some events we couldn't find the description for.

Now onto figuring out which SPN and FMI might be useful for predicting a derate. the logic here is to randomly sample rows and compare the frequency of codes present there, with respect of the frequency of codes present where a derate occured.

In [13]:
sample_codes = (
    faults_rolling
        .sample(5000)
        .drop(columns=['RecordID','EventTimeStamp','EquipmentID','spn', 'fmi'])
        .sum()
)

# 928 rows have derate as current event
derate_codes = (
    faults_rolling
        .loc[faults_rolling['spn'] == 5246]
        .drop(columns=['RecordID','EventTimeStamp','EquipmentID','spn', 'fmi'])
        .sum()
)

code_differences = (derate_codes / derate_codes.sum()) - (sample_codes / sample_codes.sum())

In [14]:
# note 5246 are all derates ... .sort_values(ascending=False).head(20)
code_differences = (
    code_differences
    .to_frame()
    .reset_index()
    .rename(columns={'index': 'spn_fmi', 0:'rel_frequency'})
)

# I did it kinda backwards; these need to be eliminated earlier
code_differences = code_differences.loc[~code_differences['spn_fmi'].str.contains('5246')]

In [15]:
code_differences.sort_values(by='rel_frequency', ascending=False).head(20)

,spn_fmi,rel_frequency
142,spn_fmi_1569_31,0.151554
368,spn_fmi_3362_31,0.033420
446,spn_fmi_4094_18,0.031975
193,spn_fmi_1761_19,0.020173
196,spn_fmi_1761_9,0.018892
379,spn_fmi_3364_9,0.016787
617,spn_fmi_5394_17,0.016127
620,spn_fmi_5394_5,0.012273
809,spn_fmi_6802_31,0.011373
279,spn_fmi_3031_9,0.010803


In [16]:
# I'm not exactly sure how to interpret this
#chi2(faults_rolling.drop(columns=['EventTimeStamp','EquipmentID','spn', 'fmi']), faults_rolling['spn'])

Using the codes above that have a positive frequency (more likely to be associated with a derate), their distributions are very close to 0. The ones that have a negative frequency means it is more likely to not be associated with a derate.

In [17]:
faults_rolling.loc[(faults_rolling['spn'] == 5246)][['spn_fmi_1569_31',
	'spn_fmi_3362_31',
    'spn_fmi_4094_18',
    'spn_fmi_1761_19',
    'spn_fmi_1761_9',
    'spn_fmi_3364_9',
    'spn_fmi_5394_17',
    'spn_fmi_5394_5',
    'spn_fmi_6802_31',
    'spn_fmi_3031_9']].describe()

#.to_csv('../data/rolling_trucks.csv')

,spn_fmi_1569_31,spn_fmi_3362_31,spn_fmi_4094_18,spn_fmi_1761_19,spn_fmi_1761_9,spn_fmi_3364_9,spn_fmi_5394_17,spn_fmi_5394_5,spn_fmi_6802_31,spn_fmi_3031_9
count,491.000000,491.000000,491.000000,491.000000,491.000000,491.000000,491.000000,491.000000,491.00000,491.000000
mean,1.004073,0.215886,0.205703,0.134420,0.124236,0.109980,0.103870,0.079430,0.07332,0.071283
std,1.554448,0.749307,0.474286,0.650162,0.625025,0.499103,0.312013,0.325458,0.54466,0.314627
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
75%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
max,12.000000,7.000000,3.000000,7.000000,7.000000,4.000000,2.000000,2.000000,7.00000,3.000000


There are 1045 trucks in the dataset, 1185166 rows; 498 have partial derail, 211 total and there's 182 with both.

Finding out below the trucks that have only partial derail, total, both, or neither.

In [18]:
all_trucks = faults['EquipmentID'].unique()
partial_derate = faults.loc[(faults['spn'] == 1569) & (faults['fmi'] == 31)]['EquipmentID'].unique()
total_derate = faults.loc[faults['spn'] == 5246]['EquipmentID'].unique()

partial_derate_only = partial_derate[np.isin(partial_derate, total_derate, invert=True)]
total_derate_only = total_derate[np.isin(total_derate, partial_derate, invert=True)]
partial_and_total_derate = np.intersect1d(partial_derate, total_derate)
no_derate = all_trucks[np.isin(all_trucks, partial_derate_only, invert=True) | np.isin(all_trucks, total_derate_only, invert=True)]



In [19]:
print(len(partial_derate_only))
print(len(total_derate_only))
print(len(partial_and_total_derate))
print(len(no_derate))

330
28
161
1042


## Creating Predictor Variable

In order to be able to train the models, we need to create a predictor variable. Essentially, look into the "future" of each row and see if a derate happens (let's say within a 6hr window).

To do that, I used a similar approach as above, when performing data exploration. Major difference here is that I sorted the timeseries backwards (in order to "look at the future") and only used spn instead of spn_fmi, because I was looking at full derates that have same spn but different possible fmi.

In [20]:
faults_encodedspn = faults.drop(columns='spn_fmi')

faults_encodedspn['spn_double'] = faults_encodedspn['spn']

# this column is so that we can sort derates at the top when the events happen at same time!
# this actually gets additional 48 rows (that are happening at same time)
faults_encodedspn['dummy_derate'] = np.where(faults_encodedspn['spn'] == 5246, 1, 0)

faults_encodedspn = pd.get_dummies(faults_encodedspn, columns=['spn_double'], prefix='spn')

# have to invert the time order here to look into the future!
faults_encodedspn = faults_encodedspn.sort_values(by=['EventTimeStamp','dummy_derate'], ascending=[False, False])

In [21]:
spn_cols = [col for col in faults_encodedspn.columns if 'spn_' in col]
fixed_cols = ['RecordID', 'spn', 'fmi']

In [22]:
faults_rolling_future = (
    faults_encodedspn
    .groupby('EquipmentID')[['EventTimeStamp'] + spn_cols]
    .rolling(window = '6h', on = "EventTimeStamp")
    .sum()
)

faults_rolling_future = faults_rolling_future.reset_index()

In [23]:
faults_rolling_future = pd.merge(faults_encodedspn[fixed_cols],
                          faults_rolling_future,
                          left_index= True,
                          right_on = 'level_1').drop(columns='level_1')

In [24]:
# if we don't include the times when derate already happened & (faults_rolling_future['spn'] != 5246)
faults_rolling_future['target'] = np.where(faults_rolling_future['spn_5246'] > 0, 1, 0)
#faults_rolling_future['target'] = np.where(faults_rolling_future['spn_1569'] > 0, 1, 0)

In [25]:
y = faults_rolling_future[['RecordID', 'target']]

### Alternative

Ajay designed a function (below) that should return the same output.

In [26]:
def  GetFilteredSPNbyDays(df_faults, windowTimeframeUnit, day_window):
    df_new = pd.DataFrame(columns = ['RecordID','EquipmentID', 'EventTimeStamp','active', 'spn', 'fmi', 'target'])  #'EventTimeStamp_DateOnly', 'PartialDerate',
    #print(df_new)
    df_new = df_new.astype({'EquipmentID': 'int'})
    dts_evt = ""
    dts_evt_max=""

    hasDerate = False
    # loop through rows of original dataframe and assign new values to columns of new dataframe
    for index, row in df_faults.iterrows():
        #if((row['spn'] == 1569) & (row['fmi'] == 31)):  
        if(row['spn'] == 5246): 
            if(hasDerate == False):
                hasDerate = True
                #df_new.loc[index, 'PartialDerate'] = 1
                df_new.loc[index, 'target'] = 0
                dts_evt = row['EventTimeStamp'] 
                #dts_evt_max = dts_evt - timedelta(days=day_window)
                if(windowTimeframeUnit == "hours"):                   
                    dts_evt_max = dts_evt - timedelta(hours=day_window)
                    #print("hours = dts_evt_max= " + str(dts_evt_max))
                elif(windowTimeframeUnit == "days"):                   
                    dts_evt_max = dts_evt - timedelta(days=day_window)
                    #print("days = dts_evt_max= " + str(dts_evt_max))
                #add cols
                df_new.loc[index, 'RecordID'] = row['RecordID']
                df_new.loc[index, 'EquipmentID'] = row['EquipmentID']
                df_new.loc[index, 'EventTimeStamp'] = row['EventTimeStamp']
                #df_new.loc[index, 'EventTimeStamp_DateOnly'] = row['EventTimeStamp_DateOnly'] 
                df_new.loc[index, 'active'] = row['active']
                df_new.loc[index, 'spn'] = row['spn']
                df_new.loc[index, 'fmi'] = row['fmi']

        else:
            if(dts_evt_max != ""):
 
                if((row['EventTimeStamp'] > dts_evt_max) & (hasDerate)) : #row['EventTimeStamp'] <= dts_evt)
                    #print('EventTimeStamp > dts_evt_max' + str(dts_evt_max) + " ---- " + str(row['EventTimeStamp_DateOnly']))
                    #df_new.loc[index, 'PartialDerate'] = 0 
                    df_new.loc[index, 'target'] = 1
                    df_new.loc[index, 'RecordID'] = row['RecordID']
                    df_new.loc[index, 'EquipmentID'] = row['EquipmentID']
                    df_new.loc[index, 'EventTimeStamp'] = row['EventTimeStamp']
                    #df_new.loc[index, 'EventTimeStamp_DateOnly'] = row['EventTimeStamp_DateOnly'] 
                    df_new.loc[index, 'active'] = row['active']
                    df_new.loc[index, 'spn'] = row['spn']
                    df_new.loc[index, 'fmi'] = row['fmi']
                else:
                    #print('ELSE  ' + str(dts_evt_max)+ " ---- " +  str(row['EventTimeStamp_DateOnly']))
                    hasDerate = False
                    dts_evt = ""
                    dts_evt_max = ""
    #print(df_new) 
    return df_new
    

In [27]:
df_filtered = pd.DataFrame()
for chunk in pd.read_csv("../data/df_faults.csv", chunksize=5000, parse_dates=['EventTimeStamp']):
    dfW = GetFilteredSPNbyDays(chunk, "hours", 6)
    #print(dfW)
    dfW['active']=dfW['active'].astype(bool) #to avoid futuretype warning
    df_filtered = pd.concat([df_filtered, dfW])

In [28]:
y['target'].sum()

1389

In [29]:
df_filtered.shape

(4057, 7)

In [30]:
test_list = df_filtered.sort_values(by='RecordID')['RecordID']

In [31]:
y[(y['target'] == 1) & (~y['RecordID'].isin(test_list))].sort_values(by='RecordID')

,RecordID,target
57,5715,1
58,5716,1
69802,6568,1
69803,6569,1
69804,6570,1
...,...,...
533396,1222021,1
533394,1223046,1
533393,1223073,1
533392,1223981,1


In [32]:
faults[faults['RecordID'] == 6568]

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,spn_fmi
6567,6568,1099612,2015-02-27 04:16:52,Low (Severity Medium) Catalyst Tank Temperature,04993120*00054166*082113134117*07700053*I0*BBZ*,79487658,6X1u10D1500000000,CMMNS,0,3031,18,True,1,1486,105338831,38.966898,-78.437638,2015-02-27 04:17:29,3031_18


In [33]:
faults[(faults['EquipmentID'] == '1486') & (faults['spn'] == 5246)]

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,spn_fmi
6628,6629,1104361,2015-02-27 09:09:56,NaN,04993120*00054166*082113134117*07700053*I0*BBZ*,79487658,6X1u10D1500000000,CMMNS,0,5246,0,True,1,1486,105338831,40.534259,-76.431805,2015-02-27 09:10:33,5246_0
6665,6666,1109134,2015-02-27 12:45:34,NaN,04993120*00054166*082113134117*07700053*I0*BBZ*,79487658,6X1u10D1500000000,CMMNS,0,5246,0,True,1,1486,105338831,41.225879,-77.074907,2015-02-27 12:46:11,5246_0
6684,6685,1114036,2015-02-27 16:52:12,NaN,04993120*00054166*082113134117*07700053*I0*BBZ*,79487658,6X1u10D1500000000,CMMNS,0,5246,0,True,1,1486,105338831,41.033333,-77.515648,2015-02-27 16:52:49,5246_0


In [34]:
y['RecordID'].nunique()

546674

In [35]:
df_filtered.shape

(4057, 7)

In [36]:
df_filtered['RecordID'].nunique()

4057

In [37]:
df_filtered.sort_values(by=['EquipmentID', 'target', 'EventTimeStamp']).head(50)

,RecordID,EquipmentID,EventTimeStamp,active,spn,fmi,target
9809,527157,301.0,2016-07-12 19:11:07,True,5246,19,0
7900,1230409,302.0,2020-01-06 10:13:57,True,5246,19,0
7897,1232200,302.0,2020-01-13 13:18:31,True,5246,19,0
7871,1242137,302.0,2020-02-14 11:21:54,True,5246,19,0
7901,1230352,302.0,2020-01-06 07:20:54,True,3556,5,1
7902,1230351,302.0,2020-01-06 07:20:54,True,1787,11,1
6662,383984,304.0,2016-02-15 10:59:28,True,5246,14,0
6675,383534,304.0,2016-02-15 06:38:41,True,3821,11,1
6674,383535,304.0,2016-02-15 06:39:20,True,1761,19,1
6673,383688,304.0,2016-02-15 08:15:31,True,3697,9,1


In [38]:
faults[(faults['EquipmentID'] == '305')   & (faults['spn'] == 5246)]

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,spn_fmi
306901,311407,6322249,2015-12-14 18:25:46,NaN,unknown,unknown,unknown,unknown,49,5246,19,True,3,305,105362919,35.196203,-85.814722,2015-12-14 18:26:23,5246_19
327279,332336,6634845,2016-01-03 17:49:35,NaN,unknown,unknown,unknown,unknown,49,5246,19,True,4,305,105362919,34.793564,-84.823333,2016-01-03 17:50:10,5246_19
359437,366938,7182877,2016-02-01 06:15:40,NaN,unknown,unknown,unknown,unknown,49,5246,19,True,1,305,105362919,34.354722,-82.941342,2016-02-01 06:16:16,5246_19
993600,1029758,53172815,2018-06-25 14:48:16,NaN,unknown,unknown,unknown,unknown,49,5246,19,True,2,305,105362919,37.644166,-85.348148,2018-06-25 14:48:54,5246_19
1021723,1060718,62965437,2018-09-26 12:37:10,NaN,unknown,unknown,unknown,unknown,49,5246,19,True,24,305,105362919,38.192083,-85.707407,2018-09-26 12:38:01,5246_19
1034642,1075522,67315847,2018-11-05 10:33:01,NaN,PC4__1284P4C_6*,Y048665,MX16U13D13,PCAR,0,5246,14,True,1,305,105362919,38.192453,-85.707824,2018-11-05 10:33:38,5246_14
1034729,1075609,67346983,2018-11-05 14:11:23,NaN,unknown,unknown,unknown,unknown,49,5246,19,True,26,305,105362919,38.192546,-85.707824,2018-11-05 14:12:51,5246_19
1036165,1077045,67784947,2018-11-08 21:55:05,NaN,unknown,unknown,unknown,unknown,49,5246,19,True,30,305,105362919,38.192314,-85.707592,2018-11-08 21:55:41,5246_19
1036168,1077048,67784950,2018-11-08 21:55:05,NaN,PC4__1284P4C_6*,Y048665,MX16U13D13,PCAR,0,5246,14,True,2,305,105362919,38.192314,-85.707592,2018-11-08 21:55:42,5246_14
1036596,1077476,67869811,2018-11-08 22:13:34,NaN,unknown,unknown,unknown,unknown,49,5246,19,True,36,305,105362919,38.192314,-85.707685,2018-11-08 23:45:38,5246_19


In [39]:
faults.loc[faults['EventTimeStamp'] == '2018-09-26 12:37:10'].sort_values(by=['EquipmentID','EventTimeStamp', 'RecordID'], ascending=[False, False, False])

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,spn_fmi
1021728,1060723,62965444,2018-09-26 12:37:10,Data May Be Invalid Catalyst Tank Level,unknown,unknown,unknown,unknown,49,1761,19,True,2,305,105362919,38.192083,-85.707407,2018-09-26 12:38:01,1761_19
1021727,1060722,62965443,2018-09-26 12:37:10,Special Instructions Maximum Vehicle Speed Limit,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,74,14,True,1,305,105362919,38.192083,-85.707407,2018-09-26 12:38:01,74_14
1021726,1060721,62965442,2018-09-26 12:37:10,High Voltage (Engine Fuel Shutoff 1 Control),PC4__1284P4C_2*,6U13D13,MX,PCAR,0,632,3,True,1,305,105362919,38.192083,-85.707407,2018-09-26 12:38:01,632_3
1021725,1060720,62965440,2018-09-26 12:37:10,Incorrect Data Ambient Air Temperature,unknown,unknown,unknown,unknown,49,171,2,True,2,305,105362919,38.192083,-85.707407,2018-09-26 12:38:01,171_2
1021724,1060719,62965439,2018-09-26 12:37:10,Abnormal Update Rate Particulate Trap Lamp Com...,unknown,unknown,unknown,unknown,49,3697,9,True,6,305,105362919,38.192083,-85.707407,2018-09-26 12:38:01,3697_9
1021723,1060718,62965437,2018-09-26 12:37:10,NaN,unknown,unknown,unknown,unknown,49,5246,19,True,24,305,105362919,38.192083,-85.707407,2018-09-26 12:38:01,5246_19
1021722,1060717,62965436,2018-09-26 12:37:10,Low Current Engine (Compression) Brake Output #1,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,1072,5,True,1,305,105362919,38.192083,-85.707361,2018-09-26 12:38:00,1072_5
1021721,1060716,62965435,2018-09-26 12:37:10,Incorrect Data J1939 Network #2,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,1231,2,True,1,305,105362919,38.192083,-85.707361,2018-09-26 12:38:00,1231_2
1021720,1060715,62965434,2018-09-26 12:37:10,Incorrect Data J1939 Network #4,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,1668,2,True,1,305,105362919,38.192083,-85.707361,2018-09-26 12:38:00,1668_2
1021719,1060714,62965433,2018-09-26 12:37:10,Error in System Engine Exhaust Gas Recirculati...,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,3821,11,True,2,305,105362919,38.192083,-85.707361,2018-09-26 12:38:00,3821_11


In [40]:
testing = pd.merge(y, df_filtered, how='inner', on='RecordID')

In [41]:
faults.loc[faults['RecordID'] == 1060713]

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,spn_fmi
1021718,1060713,62965432,2018-09-26 12:37:10,Special Instructions Catalyst Tank Level,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,1761,14,True,1,305,105362919,38.192083,-85.707361,2018-09-26 12:38:00,1761_14


In [42]:
testing.sort_values(by=['EquipmentID', 'target_x']).head(30)

,RecordID,target_x,EquipmentID,EventTimeStamp,active,spn,fmi,target_y
1173,527157,1,301.0,2016-07-12 19:11:07,True,5246,19,0
0,1242137,1,302.0,2020-02-14 11:21:54,True,5246,19,0
18,1232200,1,302.0,2020-01-13 13:18:31,True,5246,19,0
19,1230409,1,302.0,2020-01-06 10:13:57,True,5246,19,0
20,1230351,1,302.0,2020-01-06 07:20:54,True,1787,11,1
21,1230352,1,302.0,2020-01-06 07:20:54,True,3556,5,1
1852,383984,1,304.0,2016-02-15 10:59:28,True,5246,14,0
1853,383983,1,304.0,2016-02-15 10:59:26,True,5848,9,1
1854,383982,1,304.0,2016-02-15 10:59:20,True,3821,11,1
1856,383958,1,304.0,2016-02-15 10:50:42,True,1668,2,1


## Vehicle Diagnostic

For vehicle diagnostic:
- Id -  the record Id
- Name – the name of the diagnostic
- Value – the value for that diagnostic
- FaultId – foreign key to the QCJ1939Fault record

Work on the Diagnostic table done by Alison Cordoba

In [43]:
vehicle_diagnostic.head(10)

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1
5,6,AcceleratorPedal,0,1
6,7,IntakeManifoldTemperature,78.8,1
7,8,FuelRate,0,1
8,9,FuelLtd,12300.907429328,1
9,10,EngineRpm,0,1


In [51]:
# make a copy of DF to prevent accidental changes
Diagnostics = vehicle_diagnostic

In [52]:
Diagnostics = Diagnostics.pivot(index="FaultId", columns="Name", values="Value").reset_index()

In [53]:
# Remove commas from all 11 columns
for col in Diagnostics.columns[:21]:
    Diagnostics[col] = Diagnostics[col].astype(str).str.replace(',', '')

# Convert all columns to numeric
Diagnostics = Diagnostics.apply(pd.to_numeric, errors='coerce')

Diagnostics

Name,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,0.0,14.2100,NaN,66.48672,423178.70000,100.4,11.0,0.00,96.74375,...,NaN,NaN,78.8,1023,NaN,NaN,0.00000,3276.75,NaN,0.00
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
1187331,1248455,100.0,14.5000,NaN,64.62260,423937.90000,185.0,51.0,37.12,211.49370,...,32.0,NaN,98.6,18431,NaN,NaN,65.01096,NaN,73.2,7.83
1187332,1248456,0.0,14.3550,NaN,66.48672,465925.40000,186.8,62.0,41.18,212.84380,...,NaN,NaN,91.4,17407,NaN,NaN,66.57410,NaN,100.0,6.96
1187333,1248457,1.6,14.4275,NaN,67.72946,28606.65625,181.4,0.0,27.26,221.73120,...,NaN,NaN,100.4,1023,NaN,NaN,11.84489,14.10,100.0,1.74


In [54]:
Diagnostics = Diagnostics.drop(columns=['CruiseControlActive', 'IgnStatus', 'ParkingBrake'])

In [112]:
faults_improved = faults.merge(Diagnostics, left_on='RecordID', right_on='FaultId')
faults_improved

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,...,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,111,...,12300.907429,0.000000,NaN,78.8,1023,NaN,0.000000,3276.75,NaN,0.00
1,2,990360,2015-02-21 11:34:34,NaN,unknown,unknown,unknown,unknown,11,629,...,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN
2,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,...,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN
3,6,990431,2015-02-21 11:40:22,Low (Severity Low) Engine Coolant Level,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,...,70349.809964,4.583399,NaN,111.2,1023,NaN,13.602200,3276.75,NaN,6.67
4,7,990439,2015-02-21 11:40:52,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,111,...,40961.065437,14.291750,NaN,78.8,1023,NaN,41.534780,3276.75,NaN,20.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546669,1248448,123899434,2020-03-06 13:12:43,High Voltage (Fuel Level),NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,...,51466.131257,0.620806,NaN,120.2,1279,NaN,0.941766,NaN,100.0,1.16
546670,1248452,123901805,2020-03-06 13:42:48,Low (Severity Medium) Engine Coolant Level,04358814*06030918*051718174436*09401683*G1*BDR*,79904453,6X1u13D1500000000,CMMNS,0,111,...,64491.926797,0.515137,NaN,104.0,2047,NaN,5.932153,NaN,100.0,0.58
546671,1248455,123905139,2020-03-06 14:04:23,Condition Exists Engine Protection Torque Derate,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,...,58979.184416,7.647805,32.0,98.6,18431,NaN,65.010960,NaN,73.2,7.83
546672,1248456,123905996,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intak...,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,...,65080.105870,8.995086,NaN,91.4,17407,NaN,66.574100,NaN,100.0,6.96


In [113]:
cols = ['activeTransitionCount','MCTNumber', 'AcceleratorPedal',
         'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd',
         'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 
        'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 
        'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature', 'LampStatus', 
        'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure']


imputer = SimpleImputer(strategy='mean')


for column in cols:
    #print('current ', column)
    
    equipment_fixed = faults_improved.groupby('EquipmentID')[column].apply(lambda x: imputer.fit_transform(x.values.reshape(-1, 1)))

    for Id in equipment_fixed.index :
        #print('id', Id)
        
        # the flatten will turn an empty array to a single digit and crash the loop
        # meaning, we still have to impute values for trucks that don't have any single value in a column filled
        if len(equipment_fixed.loc[Id].flatten() > 0):
            faults_improved.loc[faults_improved['EquipmentID'] == Id, column] = equipment_fixed.loc[Id].flatten()

In [131]:
faults.columns

Index(['RecordID', 'ESS_Id', 'EventTimeStamp', 'eventDescription',
       'ecuSoftwareVersion', 'ecuSerialNumber', 'ecuModel', 'ecuMake',
       'ecuSource', 'spn', 'fmi', 'active', 'activeTransitionCount',
       'EquipmentID', 'MCTNumber', 'Latitude', 'Longitude',
       'LocationTimeStamp', 'spn_fmi'],
      dtype='object')

In [133]:
# dropped the columns from saving because that reduces the file size from 200mb to 80mb
# faults_improved.drop(columns=['RecordID', 'ESS_Id', 'EventTimeStamp', 'eventDescription',
#        'ecuSoftwareVersion', 'ecuSerialNumber', 'ecuModel', 'ecuMake',
#        'ecuSource', 'spn', 'fmi', 'active', 'activeTransitionCount',
#        'EquipmentID', 'MCTNumber', 'Latitude', 'Longitude',
#        'LocationTimeStamp', 'spn_fmi']).to_csv('../data/diagnostics_imputed.csv')